In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType,ArrayType,LongType
import logging
import sys
import traceback
import ast
from pyspark.sql.functions import length

import json
import os
import sys
import traceback
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr,when,to_date ,udf, concat_ws,posexplode, from_json
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, expr
from pyspark.sql.types import ArrayType, FloatType
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (
    Tokenizer,
    Normalizer,
    StopWordsCleaner,
    BertEmbeddings,
    SentenceEmbeddings,DistilBertEmbeddings
)
from sparknlp.base import DocumentAssembler, Pipeline
import numpy as np
import faiss
import os
import boto3

In [2]:
spark = SparkSession.builder \
    .appName("MinIO with Delta Lake BERT") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.cores", "6") \
    .config("spark.executor.cores", "6") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "1") \
    .config("spark.dynamicAllocation.maxExecutors", "1") \
    .config("spark.dynamicAllocation.initialExecutors", "1") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.jars", "jars/hadoop-aws-3.3.4.jar,jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/delta-core_2.12-2.2.0.jar,jars/delta-storage-2.2.0.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "conbo123") \
    .config("spark.hadoop.fs.s3a.secret.key", "123conbo") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .config("delta.enable-non-concurrent-writes", "true") \
    .config('spark.sql.warehouse.dir', "s3a://lakehouse/") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()



:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-acf371e4-e0db-4002-a6ce-f4374a9b966e;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failur

25/06/15 05:15:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [15]:
# Đọc dữ liệu từ Parquet
spark_df = spark.read.format("parquet").load("s3a://lakehouse/gold/machineDataBert")

pandas_df = spark_df.toPandas()

# Ghi ra file Excel
output_path = "./kkk/machineDataBert.xlsx"  # hoặc đường dẫn bạn muốn
pandas_df.to_excel(output_path, index=False)

print(f"✅ Đã xuất ra Excel tại: {output_path}")

✅ Đã xuất ra Excel tại: ./kkk/machineDataBert.xlsx


In [8]:
spark_df = spark.read.format("parquet").load("s3a://lakehouse/gold/machineDataBert")

In [13]:
spark_df2 = spark.read.format("parquet").load("s3a://lakehouse/gold/machineDataTest")

In [17]:
from sparknlp.base import DocumentAssembler, Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("comb_bert") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

distilbert_embeddings = DistilBertEmbeddings.pretrained("distilbert_base_uncased", "en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("bert_vec") \
    .setCaseSensitive(False) \
    .setMaxSentenceLength(512)

sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["document", "bert_vec"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

nlp_pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    distilbert_embeddings,
    sentence_embeddings
])

dummy_df = spark.createDataFrame([["dummy"]], ["comb_bert"])
nlp_pipeline_model = nlp_pipeline.fit(dummy_df)
processed_df = nlp_pipeline_model.transform(spark_df)


# nlp_pipeline_model = nlp_pipeline.fit(spark_df)
# processed_df = nlp_pipeline_model.transform(spark_df)


processed_df = processed_df.select("id", "sentence_embeddings.embeddings")
processed_df = processed_df.withColumn("vecs", col("embeddings")[0]).drop("embeddings")
processed_df.select('id', 'vecs').write.format("delta").mode("overwrite").save("s3a://lakehouse/data/bert2")





distilbert_base_uncased download started this may take some time.
Approximate size to download 235.8 MB
[OK!]
25/06/15 03:36:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [18]:
nlp_pipeline_model.write().overwrite().save("./work")

In [3]:
import os
import boto3

# Kết nối tới MinIO
s3_client = boto3.client(
    "s3",
    aws_access_key_id="conbo123",
    aws_secret_access_key="123conbo",
    endpoint_url='http://minio:9000'
)
def delete_s3_folder(bucket, s3_prefix):
    """
    Xóa tất cả các object trong bucket theo prefix (giống như xóa thư mục).
    """
    print(f"Deleting existing objects in s3://{bucket}/{s3_prefix}")
    paginator = s3_client.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=bucket, Prefix=s3_prefix)

    keys_to_delete = []
    for page in pages:
        for obj in page.get("Contents", []):
            keys_to_delete.append({"Key": obj["Key"]})

    if keys_to_delete:
        # Xoá theo batch (tối đa 1000 object mỗi lần)
        for i in range(0, len(keys_to_delete), 1000):
            batch = keys_to_delete[i:i+1000]
            s3_client.delete_objects(Bucket=bucket, Delete={"Objects": batch})
        print(f"Deleted {len(keys_to_delete)} objects.")
    else:
        print("No existing objects found.")
def upload_folder_to_s3(local_folder, bucket, s3_prefix):
    """
    Upload toàn bộ thư mục local_folder lên MinIO (hoặc S3) vào bucket/s3_prefix.
    """
    for root, _, files in os.walk(local_folder):
        for file in files:
            local_path = os.path.join(root, file)
            # Tính đường dẫn tương đối so với thư mục gốc
            relative_path = os.path.relpath(local_path, local_folder)
            # Sử dụng dấu "/" để tương thích với S3 path
            s3_key = os.path.join(s3_prefix, relative_path).replace("\\", "/")
            print(f"Uploading {local_path} to s3://{bucket}/{s3_key}")
            s3_client.upload_file(local_path, bucket, s3_key)

In [4]:
delete_s3_folder("lakehouse","bert_pipeline1000")
upload_folder_to_s3("./work", "lakehouse", "bert_pipeline1000")

Deleting existing objects in s3://lakehouse/bert_pipeline1000
No existing objects found.
Uploading ./work/metadata/.part-00000.crc to s3://lakehouse/bert_pipeline1000/metadata/.part-00000.crc
Uploading ./work/metadata/._SUCCESS.crc to s3://lakehouse/bert_pipeline1000/metadata/._SUCCESS.crc
Uploading ./work/metadata/part-00000 to s3://lakehouse/bert_pipeline1000/metadata/part-00000
Uploading ./work/metadata/_SUCCESS to s3://lakehouse/bert_pipeline1000/metadata/_SUCCESS
Uploading ./work/stages/0_DocumentAssembler_22b1808fafd8/metadata/.part-00000.crc to s3://lakehouse/bert_pipeline1000/stages/0_DocumentAssembler_22b1808fafd8/metadata/.part-00000.crc
Uploading ./work/stages/0_DocumentAssembler_22b1808fafd8/metadata/._SUCCESS.crc to s3://lakehouse/bert_pipeline1000/stages/0_DocumentAssembler_22b1808fafd8/metadata/._SUCCESS.crc
Uploading ./work/stages/0_DocumentAssembler_22b1808fafd8/metadata/part-00000 to s3://lakehouse/bert_pipeline1000/stages/0_DocumentAssembler_22b1808fafd8/metadata/par

In [5]:
from pyspark.ml import PipelineModel

nlp = PipelineModel.load("s3a://lakehouse/bert_pipeline1000")

25/06/15 05:19:08 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [16]:
import numpy as np
import faiss
from pyspark.sql.functions import col

vec_df = spark.read.format("delta").load("s3a://lakehouse/data/bert2")
rows = vec_df.select("id", "vecs").rdd.map(lambda row: (row["id"], np.array(row["vecs"], dtype=np.float32))).collect()

ids = [r[0] for r in rows]
vectors = np.stack([r[1] for r in rows])

norms = np.linalg.norm(vectors, axis=1, keepdims=True)
normalized_vectors = vectors / norms

index = faiss.IndexFlatIP(normalized_vectors.shape[1])
index.add(normalized_vectors)

print(f"✅ FAISS Cosine Index chứa {index.ntotal} vectors.")

[Stage 54:==================================================>       (7 + 1) / 8]

✅ FAISS Cosine Index chứa 20658 vectors.


In [28]:
import numpy as np
import faiss
from pyspark.sql.functions import col

# Truy vấn người dùng
user_query = "This is a story about: The world is under attack by an alien armada led by the powerful Apokoliptian, Darkseid"
query_df = spark.createDataFrame([[user_query]], ["comb_bert"])

# Lấy vector
query_vector = nlp.transform(query_df) \
    .select(col("sentence_embeddings.embeddings")[0].alias("vec")) \
    .collect()[0]["vec"]
query_vector = np.array(query_vector, dtype=np.float32)

# Normalize
query_vector /= np.linalg.norm(query_vector)
query_vector = query_vector.reshape(1, -1)

# FAISS search
k = 10
scores, indices = index.search(query_vector, k)

# In kết quả
print("🎯 Top kết quả (Cosine Similarity):")
matched_ids = []
for i, idx in enumerate(indices[0]):
    matched_ids.append(ids[idx])
    print(f"{i+1}. Movie ID: {ids[idx]}, Cosine Score: {scores[0][i]:.4f}")

# Lọc kết quả từ DataFrame
results = spark_df.filter(col("id").isin(matched_ids)) \
                  .select("id", "comb_bert")

results.head(10)

🎯 Top kết quả (Cosine Similarity):
1. Movie ID: 26342, Cosine Score: 0.9012
2. Movie ID: 140607, Cosine Score: 0.8856
3. Movie ID: 849, Cosine Score: 0.8828
4. Movie ID: 284053, Cosine Score: 0.8826
5. Movie ID: 68721, Cosine Score: 0.8824
6. Movie ID: 197624, Cosine Score: 0.8808
7. Movie ID: 52837, Cosine Score: 0.8807
8. Movie ID: 299687, Cosine Score: 0.8806
9. Movie ID: 217993, Cosine Score: 0.8769
10. Movie ID: 45162, Cosine Score: 0.8769


[Row(id=849, comb_bert="Title: Krull. This movie is titled 'Krull'. Tagline: A world light-years beyond your imagination.. Tagline repeated: A world light-years beyond your imagination.. Plot summary: A prince and a fellowship of companions set out to rescue his bride from a fortress of alien invaders who have arrived on their home planet.. This is a story about: A prince and a fellowship of companions set out to rescue his bride from a fortress of alien invaders who have arrived on their home planet.. "),
 Row(id=217993, comb_bert="Title: Justice League: War. This movie is titled 'Justice League: War'. Tagline: A time when heroes became legends.. Tagline repeated: A time when heroes became legends.. Plot summary: The world is under attack by an alien armada led by the powerful Apokoliptian, Darkseid. A group of superheroes consisting of Superman, Batman, Wonder Woman, The Flash, Green Lantern, Cyborg, and Shazam must set aside their differences and gather together to defend Earth.. Th